This jupyter notebook aims to read the data from Google Scholar csv poll into a panda dataframe to perform basic statistics about proteomics software useage by 2019. 


In [19]:
import pandas as pd 
#import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)      

# Read the poll data 

data_df = pd.read_csv("../data/poll-results-07032019.csv")

def get_pie_chart(collection): 
    """
    This function returns a pie chart from a collection of elements. 
    """
    affiliation_counts = {} 
    for affiliation in affiliations:
        token_affiliations = str(affiliation).split(";")
        for token in token_affiliations: 
            if token.strip() != 'nan':
                if token not in affiliation_counts:
                   affiliation_counts[token] = 0
                affiliation_counts[token] = affiliation_counts[token] + 1

  
    trace=go.Pie(labels=[k  for  k in  affiliation_counts.keys()],values=[k  for  k in  affiliation_counts.values()])
    return trace

data_df.head() 

,Timestamp,Username,What is your current position?,In what kind of research institute do you work?,"Which of these software do you use for your Quant Analysis. [PeptideAtlas (StPeter, Libra, XPress)]",Which of these software do you use for your Quant Analysis. [OpenMS],Which of these software do you use for your Quant Analysis. [Other],Which of these software do you use for your Quant Analysis. [MSnBase (PMID:22113085)],Which of these software do you use for your Quant Analysis. [moFF],Which of these software do you use for your Quant Analysis. [FlashLFQ (PMID:29083185)],...,Search engine and type of analysis [MSAmanda],Search engine and type of analysis [Mascot],"Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [Label-Free Differential expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [Label-Free Base line expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [TMT Differential expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [TMT Base line expression experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [SRM/MR expression Experiments]","Which type of Protein quant data you would like to see more in proteomics resources. (e.g. PeptideAtlas, ProteomicsDB, etc) [DIA/SWATH expression experiments]",In case you would like to see something else more represented in Proteomics resources let us know.,Unnamed: 66
0,2019/02/08 1:42:21 PM GMT,user1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019/02/08 1:49:09 PM GMT,user2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019/02/08 1:54:35 PM GMT,user3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019/02/08 2:13:28 PM GMT,user4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019/02/08 3:22:10 PM GMT,user5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Label Free Analysis MS1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Read the position and occupation of every participant of the poll. 

In [20]:
affiliations = data_df.iloc[:,2]
trace = get_pie_chart(affiliations)
iplot([trace])

The number of participants in the poll by type of Institution. 

In [21]:
affiliations = data_df.iloc[:,3]
trace = get_pie_chart(affiliations)     
iplot([trace])